In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize
from tqdm import tqdm
import pandas_datareader as pdr
import datetime
from numpy.linalg import inv
from math import sqrt

In [44]:
start = datetime.datetime(2004, 1, 1)
end = datetime.datetime.now()
df = pdr.get_data_fred('DFF', start, end)
df /= 100

In [45]:
# Log likelihood function of the Hull-White model

def hull_white(params, rates, dt):
    a, b, sigma = params
    n = len(rates)
    mu = b + (rates[:-1] - b) * np.exp(-a * dt)
    sigma_v = np.sqrt((sigma**2 / (2 * a)) * (1 - np.exp(-2 * a * dt)))
    log_likelihood = -0.5 * np.sum(np.log(2 * np.pi * sigma_v**2) + ((rates[1:] - mu)**2 / sigma_v**2))
    return -log_likelihood

In [46]:
rates = df.dropna().values
dt = 1 / 360  # Assuming daily data; adjust accordingly if using different frequency

# Initial parameter guesses (a, b, sigma)
initial_params = [0.1, 0.1, 0.1]

# Estimate the parameters using MLE
result = minimize(hull_white, initial_params, bounds=[(1e-6, None), (None, None), (1e-6, None)], args=(rates, dt))
a, b, sigma = result.x

# Display estimated parameters
print(f"Estimated a: {a}")
print(f"Estimated b: {b}")
print(f"Estimated sigma: {sigma}")

Estimated a: 0.07223788500553713
Estimated b: 0.045203819696213954
Estimated sigma: 0.010676367496708188


In [47]:
# Numerical approximation of the Hessian matrix and final estimation results display
estimates = np.array([a, b, sigma])
step = 1e-5 * estimates
n_params = len(estimates)

# Initialize the Hessian matrix
H = np.zeros((n_params, n_params))

# Compute the Hessian matrix using finite differences
for i in range(n_params):
    for j in range(n_params):
        delta_i = np.zeros(n_params)
        delta_j = np.zeros(n_params)
        delta_i[i] = step[i]
        delta_j[j] = step[j]
        
        f_ij = hull_white(estimates + delta_i + delta_j, rates, dt)
        f_i_neg_j = hull_white(estimates + delta_i - delta_j, rates, dt)
        f_neg_i_j = hull_white(estimates - delta_i + delta_j, rates, dt)
        f_neg_i_neg_j = hull_white(estimates - delta_i - delta_j, rates, dt)
        
        H[i, j] = (f_ij - f_i_neg_j - f_neg_i_j + f_neg_i_neg_j) / (4 * step[i] * step[j])

# Invert the Hessian matrix to get the variance-covariance matrix
vcv = inv(H)

# Extract standard errors
std_err = np.sqrt(np.diag(vcv))

# Compute t-statistics
t_stats = estimates / std_err

# Display results
print('Parameter   Estimate       Std. Err.      T-stat')
param = ['a', 'b', 'sigma']
for i in range(n_params):
    print('{0:<11} {1:>0.6f}        {2:0.6f}    {3: 0.5f}'.format(param[i],
           estimates[i], std_err[i], t_stats[i]))

Parameter   Estimate       Std. Err.      T-stat
a           0.072238        0.134898     0.53550
b           0.045204        0.064555     0.70024
sigma       0.010676        0.000087     122.86434


In [48]:
# Parameters
Zt0 = 0.987108812
Zt1 = 0.97628571
dt = 0.25
n = 4
r0 = (Zt0 / Zt1 - 1) / dt

# Initialize the interest rate lattice (17 rows, 5 columns)
lattice = np.zeros((17, n+1))
lattice[8, 0] = r0

# Function to evolve the rate (Hull-White / Vasicek model)
def drt(a, b, r, dt, sigma, dW):
    return a * (b - r) * dt + sigma * dW

# Step size for rate changes
dr_i = sigma * np.sqrt(3 * dt)

# Build the interest rate lattice
for i in range(1, n+1):
    lattice[8, i] = lattice[8, i-1] * (1 - a * dt) + a * b * dt  # Central branch
    for j in range(1, i+1):
        # Up move (positive rates in upper rows)
        lattice[8 - j, i] = lattice[8 - j + 1, i - 1] + dr_i
        # Down move (negative rates in lower rows)
        lattice[8 + j, i] = lattice[8 + j - 1, i - 1] - dr_i

# Discount factor function
def discount_factor(r, dt):
    return np.exp(-r * dt)

# Bermudan swaption parameters
K = 0.055  # Fixed rate in the example
tenor = 0.25  # Quarterly payment

# Create a placeholder for Arrow-Debreu prices (we assume they're computed in backward induction)
arrow_debreu_lattice = np.zeros_like(lattice)

# Initialize the swaption value lattice
bermudan_value = np.zeros_like(lattice)

# Backward induction for Bermudan swaption pricing

# Payoff at the last exercise date (n = 4)
for j in range(-n, n+1, 2):  # Only even rows (symmetric moves)
    row = 8 + j//2
    r = lattice[row, n]
    # Calculate the swap value S(t, omega) using the remaining swap value formula
    remaining_swap_value = (K / 4) * (1 - discount_factor(r, tenor))  # Simplified swap value
    bermudan_value[row, n] = max(remaining_swap_value, 0)  # Receiver swaption payoff

# Perform backward induction for earlier time steps
for t in range(n-1, -1, -1):  # Backward from time step n-1 to 0
    for j in range(-t, t+1, 2):  # Iterate through all states
        row = 8 + j//2
        r = lattice[row, t]
        
        # Immediate exercise value at this node
        remaining_swap_value = (K / 4) * (1 - discount_factor(r, tenor))
        exercise_value = max(remaining_swap_value, 0)  # Payoff if exercised
        
        # Continuation value using backward induction with Arrow-Debreu prices
        continuation_value = 0
        for k in [-1, 0, 1]:  # Up, No move, Down moves
            next_row = row + k
            if 0 <= next_row < len(lattice):
                ad_next = discount_factor(lattice[next_row, t+1], dt)  # Approximating AD prices as discount factor
                bermudan_next = bermudan_value[next_row, t+1]
                continuation_value += ad_next * bermudan_next
        
        # Bermudan option value is the maximum of immediate exercise or continuation value
        bermudan_value[row, t] = max(exercise_value, continuation_value)

# The value of the Bermudan swaption is at the initial node (Bermudan swaption value at t = 0)
bermudan_swaption_value = bermudan_value[8, 0]
print(f"Bermudan Swaption Value: {bermudan_swaption_value}")


Bermudan Swaption Value: 0.0060520454072987475


In [49]:
lattice

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.08132802],
       [0.        , 0.        , 0.        , 0.07208201, 0.07209754],
       [0.        , 0.        , 0.06283601, 0.06285153, 0.06286678],
       [0.        , 0.05359   , 0.05360553, 0.05362078, 0.05363575],
       [0.04434399, 0.04435952, 0.04437477, 0.04438974, 0.04440444],
       [0.        , 0.03509799, 0.03511352, 0.03512876, 0.03514374],
       [0.        , 0.        , 0.02585198, 0.02586751, 0.02588276],
       [0.        , 0.        , 0.        , 0.01660598, 0.01662151],
       [0.        , 0.        , 0.        , 0.        , 0.00735997],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.